In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

# I. Classification

Dans une première partie, nous allons nous intéresser à la classification. On va importer des datasets de sklearn et entraîner notre modèle de XGBOOST sur de la classification binaire et multi-classes et le comparer avec deux modèles d'arbres de décision.

### I.1 Importer les datasets

In [2]:
from sklearn.datasets import load_iris, fetch_covtype, load_breast_cancer  
from sklearn.model_selection import train_test_split

# Charger un jeu de données Iris pour classification

cancer = load_breast_cancer()   # Pour la classification binaire
X, Y = cancer.data, cancer.target

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((455, 30), (114, 30))

### I.2 Entraîner deux modèles d'arbres de décision

In [3]:
from   sklearn.tree    import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from   sklearn.metrics import accuracy_score, f1_score
from   sklearn         import tree
import timeit

Xchoix_train = X_train
Ychoix_train = Y_train
Xchoix_test  = X_test
Ychoix_test  = Y_test
# fnames       = iris.columns


gini_stats          = []
entropy_stats       = []
randForest_stats       = []
gini_classifieur    = DecisionTreeClassifier(criterion='gini'   , random_state=42)
entropy_classifieur = DecisionTreeClassifier(criterion='entropy', random_state=42)

# ============ GINI ====================
# ............ Entraînement ............
temps_debut = timeit.default_timer()
gini_classifieur.fit(Xchoix_train, Ychoix_train)
gini_stats.append(timeit.default_timer() - temps_debut)
# ..... Evaluation entrainement ........
Ychoix_pred = gini_classifieur.predict(Xchoix_train)
gini_stats.append(accuracy_score(Ychoix_train, Ychoix_pred))
gini_stats.append(f1_score(Ychoix_train, Ychoix_pred, average='micro'))
# ................ Test ................
temps_debut = timeit.default_timer()
Ychoix_pred = gini_classifieur.predict(Xchoix_test)
gini_stats.append(timeit.default_timer() - temps_debut)
# ........... Evaluation test ...........
gini_stats.append(accuracy_score(Ychoix_test, Ychoix_pred))
gini_stats.append(f1_score(Ychoix_test, Ychoix_pred, average='micro'))

# =========== Entropy ==================
# ............ Entraînement ............
temps_debut = timeit.default_timer()
entropy_classifieur.fit(Xchoix_train, Ychoix_train)
entropy_stats.append(timeit.default_timer() - temps_debut)
# ..... Evaluation entrainement ........
Ychoix_pred = entropy_classifieur.predict(Xchoix_train)
entropy_stats.append(accuracy_score(Ychoix_train, Ychoix_pred))
entropy_stats.append(f1_score(Ychoix_train, Ychoix_pred, average='micro'))
# ................ Test ................
temps_debut = timeit.default_timer()
Ychoix_pred = entropy_classifieur.predict(Xchoix_test)
entropy_stats.append(timeit.default_timer() - temps_debut)
# ........... Evaluation test ...........
entropy_stats.append(accuracy_score(Ychoix_test, Ychoix_pred))
entropy_stats.append(f1_score(Ychoix_test, Ychoix_pred, average='micro'))

# =========== Random forest ==================
# ............ Entraînement ............
temps_debut = timeit.default_timer()
classifieur = RandomForestClassifier(n_estimators=100)
classifieur.fit(Xchoix_train, Ychoix_train)
randForest_stats.append(timeit.default_timer() - temps_debut)
# ..... Evaluation entrainement ........
Ychoix_pred = classifieur.predict(Xchoix_train)
randForest_stats.append(accuracy_score(Ychoix_train, Ychoix_pred))
randForest_stats.append(f1_score(Ychoix_train, Ychoix_pred, average='micro'))
# ................ Test ................
temps_debut = timeit.default_timer()
Ychoix_pred = classifieur.predict(Xchoix_test)
randForest_stats.append(timeit.default_timer() - temps_debut)
# ........... Evaluation test ...........
randForest_stats.append(accuracy_score(Ychoix_test, Ychoix_pred))
randForest_stats.append(f1_score(Ychoix_test, Ychoix_pred, average='micro'))

print('Fin')

Fin


### I.3 Entraîner un modèle XGBOOST

In [4]:
import xgboost as xgb

# Définir les paramètres du modèle
# params = {
#     'objective': 'multi:softmax',  # Fonction objective pour la classification multi-classe
#     'num_class': 7,  # Nombre de classes dans le jeu de données 
#     'max_depth': 4  # Profondeur maximale de l'arbre
# }
# params = {
#     'objective': 'binary:logistic',  # Fonction objective pour la classification binaire
#     'num_class': 2, 
#     'max_depth': 3
# }
XGBOOST_stats = []

# Créer le modèle de classification (avec les paramètres par défaut)
clf = xgb.XGBClassifier()


# ================= Entraînement =====================
temps_debut = timeit.default_timer()
# Entraîner le modèle
clf.fit(X_train, Y_train)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)
Y_pred = clf.predict(X_train)
XGBOOST_stats.append(accuracy_score(Y_train, Y_pred))
XGBOOST_stats.append(f1_score(Y_train, Y_pred, average='micro'))

# ===================== Test ========================
temps_debut = timeit.default_timer()
# Faire des prédictions sur l'ensemble de test
Y_pred = clf.predict(X_test)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)
XGBOOST_stats.append(accuracy_score(Y_test, Y_pred))
XGBOOST_stats.append(f1_score(Y_test, Y_pred, average='micro'))

In [5]:
pd.DataFrame({
    'Criteres' : ['Temps Entrainement', 'Accuracy Entrainement', 'F1 Entrainement', 'Temps Test', 'Accuracy Test', 'F1 Test'],
    'Entropie' : entropy_stats,
    'Gini'     : gini_stats,
    'XGBOOST'  : XGBOOST_stats,
    'Random Forest': randForest_stats
})

,Criteres,Entropie,Gini,XGBOOST,Random Forest
0,Temps Entrainement,0.016999,0.018650,0.118002,0.485955
1,Accuracy Entrainement,1.000000,1.000000,1.000000,1.000000
2,F1 Entrainement,1.000000,1.000000,1.000000,1.000000
3,Temps Test,0.000294,0.000323,0.001451,0.013052
4,Accuracy Test,0.947368,0.947368,0.956140,0.964912
5,F1 Test,0.947368,0.947368,0.956140,0.964912


# II. Régression

Dans cette seconde partie, nous allons entraîner un modèle de XGBOOST pour la régression et le comparer avec le modèle de régression linéaire en utilisant une dataset de Sklearn comportant les prix des maisons de California.

### II.1 Importer la dataset

In [6]:
from sklearn.datasets import fetch_california_housing

houses = fetch_california_housing()
X, Y = houses.data, houses.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((16512, 8), (4128, 8))

### II.2 Entraîner un modèle de régression linéaire

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline      import make_pipeline
from sklearn.metrics import mean_squared_error

lineaire_stats = []

reg_lineaire = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

temps_debut = timeit.default_timer()
# Entraîner le modèle de régression linéaire
reg_lineaire.fit(X_train, Y_train)
lineaire_stats.append(timeit.default_timer() - temps_debut)

temps_debut = timeit.default_timer()
# Prédire
Y_pred_lineaire = reg_lineaire.predict(X_test)
lineaire_stats.append(timeit.default_timer() - temps_debut)

# Calculer l'erreur quadratique moyenne (MSE)
mse_lineaire = mean_squared_error(Y_test, Y_pred_lineaire)
lineaire_stats.append(mse_lineaire)
print(f"MSE: {mse_lineaire}")

MSE: 0.5558915986952461


### II.3 Entraîner un modèle XGBOOST

In [8]:
# Définir les paramètres du modèle
params = {
    'objective': 'reg:squarederror',  # Fonction objective pour la régression
    'max_depth': 3  # Profondeur maximale de l'arbre
}

XGBOOST_stats = []

# Créer le modèle de régression XGBOOST avec les paramètres, changer la profondeur change le MSE, plus profond mieux c'est mais prend plus de temps
reg_XGBOOST = xgb.XGBRegressor(**params)

temps_debut = timeit.default_timer()
# Entraîner le modèle
reg_XGBOOST.fit(X_train, Y_train)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)

temps_debut = timeit.default_timer()
# Faire des prédictions sur l'ensemble de test
Y_pred_XGBOOST = reg_XGBOOST.predict(X_test)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)

# Calculer l'erreur quadratique moyenne (MSE)
mse_XGBOOST = mean_squared_error(Y_test, Y_pred_XGBOOST)
XGBOOST_stats.append(mse_XGBOOST)
print(f"MSE: {mse_XGBOOST}")


MSE: 0.2509968715591795


In [9]:
pd.DataFrame({
    'Criteres' : ['Temps Entrainement','Temps Test', 'MSE'],
    'Linéaire' : lineaire_stats,
    'XGBOOST'  : XGBOOST_stats
})

,Criteres,Linéaire,XGBOOST
0,Temps Entrainement,0.033214,0.152300
1,Temps Test,0.001046,0.003874
2,MSE,0.555892,0.250997


## **IV. Exploration des Hyperparamètres**

Dans cette section, nous examinerons plusieurs Hyperparamètres importants et comprendrons leurs fonctions

In [10]:
bookings = pd.read_csv('data/hotel_bookings.csv')
bookings.head()

,is_canceled,lead_time,arrival_date_week_number,arrival_date_day_of_month,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,deposit_type_No_Deposit,deposit_type_Non_Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,342,27,1,7,0,0,2,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,737,27,1,7,0,0,2,0.0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,7,27,1,7,0,1,1,0.0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,13,27,1,7,0,1,1,0.0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,14,27,1,7,0,2,2,0.0,0,...,0,0,0,1,0,0,0,0,1,0


In [11]:
# Define X and y
X, y = bookings.iloc[:,1:], bookings.iloc[:,0]

In [12]:
# Train and test split using sklearn
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=.33, random_state=123)

# Instatiate a XGBClassifier 
xgb_clf = xgb.XGBClassifier(random_state=123)

# Inspect the parameters
xgb_clf.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 123,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [13]:
# Fit it to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

In [14]:
# Calculate the accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

# Print the baseline accuracy
print("Baseline accuracy:", accuracy)

Baseline accuracy: 0.8382308083375699


In [15]:
from sklearn.metrics import accuracy_score

# Instantiate the XGBClassifier with 25 boosting rounds
xgb_clf = xgb.XGBClassifier(n_estimators=25, random_state=123)

### **Max depth**

_Depuis la documentation de XGBoost :_
> Profondeur maximale d'un arbre. Augmenter cette valeur rendra le modèle plus complexe et plus susceptible de surajuster.

<p align="center">
<img src="https://github.com/datacamp/Machine-Learning-With-XGboost-live-training/blob/master/assets/max_depth.png?raw=true" width = "35%"> 
</p>

Voyons ce qui se passe lorsque nous augmentons la valeur de `max_depth` de 6 à 20.


In [16]:
# Set max_depth to 10
xgb_clf.set_params(max_depth=20)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.853329944077275

### **colsample_bytree**

_Depuis la documentation de XGBoost :_

> Le taux de sous-échantillonnage des colonnes lors de la construction de chaque arbre. Le sous-échantillonnage se produit une fois pour chaque arbre construit.

Essentiellement, cela nous permet de limiter le nombre de colonnes utilisées lors de la construction de chaque arbre. Cela ajoute de l'aléatoire, rendant le modèle plus robuste au bruit. La valeur par défaut est 1 (c'est-à-dire toutes les colonnes), essayons une valeur plus petite.

<p align="center">
<img src="https://github.com/datacamp/Machine-Learning-With-XGboost-live-training/blob/master/assets/colsample_bytree.gif?raw=true" width = "55%"> 
</p>

In [17]:
# Set colsample_bytree to 0.5 
xgb_clf.set_params(colsample_bytree=0.5)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.8552872394509405

Nous pouvons également limiter le nombre de colonnes utilisées à chaque niveau de profondeur ou nœud de notre arbre.

_Selon la documentation de XGBoost :_

> `colsample_bylevel` est le taux de sous-échantillonnage des colonnes pour chaque niveau. Le sous-échantillonnage se produit une fois pour chaque nouveau niveau de profondeur atteint dans un arbre. Les colonnes sont sous-échantillonnées à partir de l'ensemble des colonnes choisies pour l'arbre en cours.

> `colsample_bynode` est le taux de sous-échantillonnage des colonnes pour chaque nœud (division). Le sous-échantillonnage se produit chaque fois qu'une nouvelle division est évaluée. Les colonnes sont sous-échantillonnées à partir de l'ensemble des colonnes choisies pour le niveau en cours.

### **subsample**

_Selon la documentation de XGBoost :_

> - Taux de sous-échantillonnage des instances d'entraînement. Le régler sur 0.5 signifie que XGBoost échantillonnera de manière aléatoire la moitié des données d'entraînement avant de développer les arbres, ce qui permettra d'éviter le surajustement. 
> - Le sous-échantillonnage se produira une fois à chaque itération.
> - range: (0,1]

La valeur par défaut est 1, essayons 0.75.

Cela signifie que chacun de nos 25 arbres recevra un échantillonnage aléatoire de 75 % de nos données d'entraînement. Chaque arbre s'entraînera sur différentes parties des données, ce qui ajoute de l'aléatoire (similaire à `colsample_bytree`). 

<p align="center">
<img src="https://github.com/datacamp/Machine-Learning-With-XGboost-live-training/blob/master/assets/subsample.gif?raw=true" width = "55%"> 
</p>

Cependant, nous ne voulons pas que cette valeur soit trop basse si nous n'avons pas beaucoup d'arbres, car notre modèle risquerait de ne pas voir suffisamment de données et de ne pas s'ajuster correctement.

In [18]:
# Set subsample to 0.75 
xgb_clf.set_params(subsample=0.75)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.8501016776817488

### **gamma**

_Selon la documentation de XGBoost :_
> - Perte minimale requise pour effectuer une partition supplémentaire sur un nœud feuille de l'arbre. Plus gamma est grand, plus l'algorithme sera conservateur.
> - range: [0,∞]


Cela décide si un nœud se scindera en fonction de la réduction de perte attendue après la division. `gamma` représente la perte minimale requise pour qu'un nœud se scinde.

<p align="center">
<img src="https://github.com/datacamp/Machine-Learning-With-XGboost-live-training/blob/master/assets/gamma.png?raw=true" width = "55%"> 
</p>

Augmenter `gamma` = moins de divisions = moins de complexité

La valeur par défaut est 0, donc dans notre cas, les nœuds se sont toujours divisés jusqu'à la profondeur maximale. Augmentons cela à 0.25.

In [19]:
# Set gamma to .25 
xgb_clf.set_params(gamma=0.25)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.850127097102186

### **Learning Rate (eta)**

_Selon la documentation de XGBoost :_
> - Taux de réduction utilisé dans la mise à jour pour prévenir le surajustement. Après chaque étape d'amplification, nous pouvons obtenir directement les poids des nouvelles fonctionnalités, et eta réduit les poids des fonctionnalités pour rendre le processus de boosting plus conservateur.
> - range: [0,1]

Le taux d'apprentissage affecte la rapidité avec laquelle un modèle apprend.

Le boosting gradient fonctionne en ajoutant séquentiellement des faibles apprenants au modèle. Chaque nouveau faible apprenant tente de corriger les erreurs résiduelles des arbres précédents. Cela rend le modèle très susceptible de surajustement. Le taux d'apprentissage peut aider à ralentir l'apprentissage en réduisant les poids résultants de l'arbre actuel avant de les transmettre à l'arbre suivant.

Le taux d'apprentissage actuel de notre modèle est de 0.1. Que se passe-t-il si nous le changeons à 0.3 ?

In [20]:
# Set learning rate to .3 
xgb_clf.set_params(learning_rate=0.3)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.850127097102186

Le taux d'apprentissage et le nombre d'arbres doivent être ajustés ensemble. Si nous diminuons le taux d'apprentissage, nous devons nous assurer d'avoir suffisamment d'arbres pour apprendre quelque chose et éviter un sous-ajustement sévère. **Par conséquent, un faible taux d'apprentissage nécessitera davantage de cycles de boosting.**

### **reg_alpha**

_Selon la documentation de XGBoost :_
> Terme de régularisation L1 sur les poids. Augmenter cette valeur rendra le modèle plus conservateur.

L1 est souvent appelé **régression Lasso**. Il s'agit d'une technique de régularisation fondamentale, ce qui signifie qu'elle vise à réduire le surajustement en décourageant les modèles complexes. Dans le cas du boosting gradient, L1 le fait en ajoutant des pénalités sur les poids des feuilles. Augmenter alpha conduit les poids des feuilles des apprenants de base vers 0.

La valeur par défaut est 0, ce qui signifie qu'il n'y a actuellement aucune régularisation alpha dans notre modèle. Activons L1 avec une valeur de `0.01`.

In [21]:
# set reg_alpha to .1 
xgb_clf.set_params(reg_alpha=0.01)

# Fit the classifier to the training set
xgb_clf.fit(X_train, y_train)

# Predict the labels of the test set
preds = xgb_clf.predict(X_test)

# Compute the accuracy
accuracy_score(y_test, preds)

0.850127097102186

**L2**, également connue sous le nom de régression ridge, est également disponible avec le paramètre reg_lambda. L2 est réputée pour avoir une pénalité plus douce que L1. Cela signifie que les poids des feuilles diminuent de manière plus régulière, avec moins de risque de parcimonie dans les poids des feuilles. Assurez-vous donc d'essayer différentes techniques de régularisation !

Examinons les paramètres résultants après les avoir modifiés manuellement.

In [22]:
# Print the model parameters
xgb_clf.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.5,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0.25,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.3,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 20,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 25,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 123,
 'reg_alpha': 0.01,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.75,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

Il existe de nombreuses combinaisons possibles de paramètres. Nous ne pouvons pas les ajuster manuellement et les choisir.